In [1]:
import torch
import torch.nn as nn
import pandas as pd
from src.models.model_structure import BasicBlock, customResNet18
import argparse
from src.utils.configer import Configer
from pathlib import Path

In [2]:
import json

logs_dir = Path("./train_logs/")

log_names = ["customResNet18_Adam_ver1", "customResNet18_Adam_ver2"]
model_print = []
for idx, name in enumerate(log_names):
    with open(logs_dir / ("train_log_" + name + ".json"), "r") as f:
        data = json.load(f)
        model_print.append(data.get("model_architecture"))

In [1]:
import torch
if torch.cuda.is_available():
    print("CUDA is available!")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version used by PyTorch: {torch.version.cuda}")
else:
    print("CUDA is not available. PyTorch is using CPU.")

CUDA is available!
Number of GPUs: 1
Current GPU Name: NVIDIA GeForce RTX 3060
CUDA Version used by PyTorch: 12.6


In [4]:
parser = argparse.ArgumentParser()

parser.add_argument('--disable-cuda', action='store_true',
                    help='Disable CUDA')
parser.add_argument('--hypes', default=None, type=str,
                    dest='hypes', help='The file of the hyper parameters.')
parser.add_argument('--phase', default='train', type=str,
                    dest='phase', help='The phase of module.')
parser.add_argument('--gpu', default=[0, ], nargs='+', type=int,
                    dest='gpu', help='The gpu used.')
parser.add_argument('--resume', default=None, type=str,
                    dest='resume', help='The path of pretrained model.')

args, unknown = parser.parse_known_args()

args.hypes = 'src/hyperparameters/config.json'
configer = Configer(args)

configer.device = configer.get("device").lower() if torch.cuda.is_available() else 'cpu'


In [ ]:
seed = 1991
torch.manual_seed(seed)

selected_classes = configer.get('dataset', 'selected_classes')
n_classes = len(selected_classes )
# RGB.
img_size = configer.get('dataset').get('img_size')

model = customResNet18(
            num_classes = n_classes,
            layers_config = configer.get("model", "layers_num")*[configer.get("model", "block_size")],
            activation = configer.get("model", "activation").lower(),
            in_channels = img_size[0],
            layer0_channels = 256 // 2**(configer.get("model", "layers_num") - 1)
        )

test_input = torch.randn(1, img_size[0], img_size[1], img_size[2])
print(configer.get("model", "layers_num")*[configer.get("model", "block_size")])
model_size = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model size: {model_size}")
model(test_input)

[2, 2, 2, 2]
Model size: 2802538
input: torch.Size([1, 3, 64, 64])
conv1: torch.Size([1, 32, 32, 32])
bn1: torch.Size([1, 32, 32, 32])
activation: torch.Size([1, 32, 32, 32])
maxpool: torch.Size([1, 32, 32, 32])
layer: torch.Size([1, 32, 16, 16])
layer: torch.Size([1, 64, 8, 8])
layer: torch.Size([1, 128, 4, 4])
layer: torch.Size([1, 256, 2, 2])
avgpool: torch.Size([1, 256, 1, 1])
flatten: torch.Size([1, 256])
output: torch.Size([1, 10])


tensor([[-0.4880, -0.1715,  0.6662, -0.5652,  0.0228,  0.4182, -0.0158, -0.9258,
         -0.2254,  0.2788]], grad_fn=<AddmmBackward0>)

In [6]:
import torch
from torchviz import make_dot
from torchsummary import summary

# Assuming you have defined `model1 = customResNet18()` and `model2 = customResNet()`
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("=== customResNet18 ===")
summary(model.to(device), (3, 64, 64))

=== customResNet18 ===
input: torch.Size([2, 3, 64, 64])
conv1: torch.Size([2, 32, 32, 32])
bn1: torch.Size([2, 32, 32, 32])
activation: torch.Size([2, 32, 32, 32])
maxpool: torch.Size([2, 32, 32, 32])
layer: torch.Size([2, 32, 16, 16])
layer: torch.Size([2, 64, 8, 8])
layer: torch.Size([2, 128, 4, 4])
layer: torch.Size([2, 256, 2, 2])
avgpool: torch.Size([2, 256, 1, 1])
flatten: torch.Size([2, 256])
output: torch.Size([2, 10])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]           4,704
       BatchNorm2d-2           [-1, 32, 32, 32]              64
              ReLU-3           [-1, 32, 32, 32]               0
         MaxPool2d-4           [-1, 32, 32, 32]               0
            Conv2d-5           [-1, 32, 16, 16]           1,024
       BatchNorm2d-6           [-1, 32, 16, 16]              64
            Conv2d-7           [-1, 32, 16, 16]        

In [ ]:
x = torch.randn(1, 3, 64, 64)  # dummy input
y = model(x)
dot = make_dot(y, params=dict(model.named_parameters()))
dot.render("customResNet18", format="png")  # saves as `customResNet18.png`

input: torch.Size([1, 3, 64, 64])
conv1: torch.Size([1, 32, 32, 32])
bn1: torch.Size([1, 32, 32, 32])
activation: torch.Size([1, 32, 32, 32])
maxpool: torch.Size([1, 32, 32, 32])
layer: torch.Size([1, 32, 16, 16])
layer: torch.Size([1, 64, 8, 8])
layer: torch.Size([1, 128, 4, 4])
layer: torch.Size([1, 256, 2, 2])
avgpool: torch.Size([1, 256, 1, 1])
flatten: torch.Size([1, 256])
output: torch.Size([1, 10])


'customResNet18.png'